In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'geum_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2015.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/장계_2015.xlsx
data/geum/자동/장계_2

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.6563 - disc_loss: 0.3789 - rmse: 0.3614 - val_loss: 0.3437
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.3684 - disc_loss: 0.3617 - rmse: 0.3486 - val_loss: 0.6943
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5694 - disc_loss: 0.3300 - rmse: 0.3765 - val_loss: 0.3632
Epoch 53/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.6075 - disc_loss: 0.3348 - rmse: 0.3902 - val_loss: 0.3512
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7862 - disc_loss: 0.3560 - rmse: 0.3406 - val_loss: 0.2722
Epoch 55/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 21.7522 - disc_loss: 0.3773 - rmse: 0.4429 - val_loss: 0.6120
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 35.2144 - disc_loss: 0.3486 - rmse: 0.6440 - val_loss: 0.3441
Epoch 57/2000
1/1 [==================

Epoch 109/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2759 - disc_loss: 0.2774 - rmse: 0.2788 - val_loss: 0.3038
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3675 - disc_loss: 0.2902 - rmse: 0.2900 - val_loss: 0.6053
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3746 - disc_loss: 0.2919 - rmse: 0.2684 - val_loss: 0.4558
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8097 - disc_loss: 0.2693 - rmse: 0.2646 - val_loss: 0.2871
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1817 - disc_loss: 0.2780 - rmse: 0.2398 - val_loss: 0.2680
Epoch 114/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9064 - disc_loss: 0.2780 - rmse: 0.2247 - val_loss: 0.3317
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8242 - disc_loss: 0.2817 - rmse: 0.2879 - val_loss: 0.2841
Epoch 116/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3049 - disc_loss: 0.2434 - rmse: 0.2123 - val_loss: 0.2401
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7010 - disc_loss: 0.2632 - rmse: 0.2431 - val_loss: 0.2557
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7608 - disc_loss: 0.2656 - rmse: 0.2597 - val_loss: 0.2358
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0462 - disc_loss: 0.2530 - rmse: 0.2741 - val_loss: 0.2982
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5158 - disc_loss: 0.2512 - rmse: 0.2492 - val_loss: 0.2393
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5611 - disc_loss: 0.2433 - rmse: 0.2455 - val_loss: 0.2001
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4076 - disc_loss: 0.2670 - rmse: 0.2432 - val_loss: 0.2652
Epoch 174/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 12.7080 - disc_loss: 0.2343 - rmse: 0.3967 - val_loss: 0.5106
Epoch 226/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6310 - disc_loss: 0.2403 - rmse: 0.2569 - val_loss: 0.2214
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1204 - disc_loss: 0.2333 - rmse: 0.3093 - val_loss: 0.2078
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 15.3396 - disc_loss: 0.2630 - rmse: 0.4049 - val_loss: 0.2569
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3842 - disc_loss: 0.2448 - rmse: 0.2481 - val_loss: 0.2637
Epoch 230/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.8528 - disc_loss: 0.2465 - rmse: 0.2349 - val_loss: 0.2088
Epoch 231/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.6286 - disc_loss: 0.2595 - rmse: 0.2502 - val_loss: 0.2483
Epoch 232/2000
1/1 [=============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.3624 - disc_loss: 0.1693 - rmse: 0.2480 - val_loss: 0.2644
Epoch 1328/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4599 - disc_loss: 0.1641 - rmse: 0.2235 - val_loss: 0.2247
Epoch 1329/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4133 - disc_loss: 0.1643 - rmse: 0.2033 - val_loss: 0.1999
Epoch 1330/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6748 - disc_loss: 0.1592 - rmse: 0.2014 - val_loss: 0.2153
Epoch 1331/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.2160 - disc_loss: 0.1670 - rmse: 0.2232 - val_loss: 0.2318
Epoch 1332/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0887 - disc_loss: 0.1719 - rmse: 0.2122 - val_loss: 0.2091
Epoch 1333/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8066 - disc_loss: 0.1635 - rmse: 0.2683 - val_loss: 0.2437
Epoch 1334/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6729 - disc_loss: 0.1629 - rmse: 0.2135 - val_loss: 0.2182
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2490 - disc_loss: 0.1644 - rmse: 0.2179 - val_loss: 0.3194
Epoch 1387/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0104 - disc_loss: 0.1535 - rmse: 0.2339 - val_loss: 0.2749
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1865 - disc_loss: 0.1622 - rmse: 0.2290 - val_loss: 0.1932
Epoch 1389/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5418 - disc_loss: 0.1613 - rmse: 0.2241 - val_loss: 0.2388
Epoch 1390/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5511 - disc_loss: 0.1553 - rmse: 0.2288 - val_loss: 0.4329
Epoch 1391/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3952 - disc_loss: 0.1756 - rmse: 0.2158 - val_loss: 0.2077
Epoch 1392/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1593 - disc_loss: 0.1611 - rmse: 0.2058 - val_loss: 0.3575
Epoch 1444/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7613 - disc_loss: 0.1646 - rmse: 0.2441 - val_loss: 0.2478
Epoch 1445/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 13.9967 - disc_loss: 0.1710 - rmse: 0.3186 - val_loss: 0.2263
Epoch 1446/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9663 - disc_loss: 0.1740 - rmse: 0.2066 - val_loss: 0.1867
Epoch 1447/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.3491 - disc_loss: 0.1655 - rmse: 0.2330 - val_loss: 0.2120
Epoch 1448/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.1075 - disc_loss: 0.1696 - rmse: 0.2808 - val_loss: 0.2454
Epoch 1449/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7310 - disc_loss: 0.1670 - rmse: 0.2018 - val_loss: 0.1959
Epoch 1450/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1622 - disc_loss: 0.1627 - rmse: 0.2469 - val_loss: 0.1814
Epoch 1502/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.6459 - disc_loss: 0.1572 - rmse: 0.2679 - val_loss: 0.2048
Epoch 1503/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4089 - disc_loss: 0.1583 - rmse: 0.2085 - val_loss: 0.1985
Epoch 1504/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9749 - disc_loss: 0.1556 - rmse: 0.1781 - val_loss: 0.3470
Epoch 1505/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5607 - disc_loss: 0.1522 - rmse: 0.1893 - val_loss: 0.3093
Epoch 1506/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6494 - disc_loss: 0.1554 - rmse: 0.1942 - val_loss: 0.1813
Epoch 1507/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.5699 - disc_loss: 0.1625 - rmse: 0.3020 - val_loss: 0.2276
Epoch 1508/2000
1/1 [=====

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1240 - disc_loss: 0.1634 - rmse: 0.2194 - val_loss: 0.1995
Epoch 1560/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2786 - disc_loss: 0.1618 - rmse: 0.2480 - val_loss: 0.2877
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8410 - disc_loss: 0.1620 - rmse: 0.2377 - val_loss: 0.2123
Epoch 1562/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8149 - disc_loss: 0.1564 - rmse: 0.2853 - val_loss: 0.1961
Epoch 1563/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8055 - disc_loss: 0.1504 - rmse: 0.1952 - val_loss: 0.1892
Epoch 1564/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6341 - disc_loss: 0.1658 - rmse: 0.1967 - val_loss: 0.2007
Epoch 1565/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6793 - disc_loss: 0.1612 - rmse: 0.2096 - val_loss: 0.2037
Epoch 1566/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [31]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(52608, 36)
toc


In [32]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


In [33]:
#fake_df.shape[0]/8760

In [34]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [37]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [38]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [39]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/toc/


In [40]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.4385 - mean_absolute_error: 0.5275 - nse: -0.0184
Epoch 00001: val_nse improved from -inf to 0.65599, saving model to save/best_model.h5
10/10 [==============================] - 8s 798ms/step - loss: 0.4385 - mean_absolute_error: 0.5275 - nse: -0.0184 - val_loss: 0.1161 - val_mean_absolute_error: 0.2670 - val_nse: 0.6560
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.1518 - mean_absolute_error: 0.3076 - nse: 0.6527
Epoch 00002: val_nse improved from 0.65599 to 0.72457, saving model to save/best_model.h5
10/10 [==============================] - 8s 783ms/step - loss: 0.1518 - mean_absolute_error: 0.3076 - nse: 0.6527 - val_loss: 0.0931 - val_mean_absolute_error: 0.2410 - val_nse: 0.7246
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.0940 - mean_absolute_error: 0.2320 - nse: 0.7768
Epoch 00003: val_nse did not improve from 0.72457
10/10 [==============================] - 8s 

10/10 [==============================] - ETA: 0s - loss: 0.0440 - mean_absolute_error: 0.1500 - nse: 0.8988
Epoch 00022: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 795ms/step - loss: 0.0440 - mean_absolute_error: 0.1500 - nse: 0.8988 - val_loss: 0.0584 - val_mean_absolute_error: 0.1751 - val_nse: 0.8281
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0477 - mean_absolute_error: 0.1561 - nse: 0.8926
Epoch 00023: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 792ms/step - loss: 0.0477 - mean_absolute_error: 0.1561 - nse: 0.8926 - val_loss: 0.0578 - val_mean_absolute_error: 0.1683 - val_nse: 0.8295
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0469 - mean_absolute_error: 0.1504 - nse: 0.8966
Epoch 00024: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 785ms/step - loss: 0.0469 - mean_absolute_error: 0.1504 - nse: 0.8966 - val_loss: 0.0546

10/10 [==============================] - ETA: 0s - loss: 0.0347 - mean_absolute_error: 0.1320 - nse: 0.9205
Epoch 00045: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 798ms/step - loss: 0.0347 - mean_absolute_error: 0.1320 - nse: 0.9205 - val_loss: 0.0966 - val_mean_absolute_error: 0.2123 - val_nse: 0.7161
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.0343 - mean_absolute_error: 0.1350 - nse: 0.9173
Epoch 00046: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 770ms/step - loss: 0.0343 - mean_absolute_error: 0.1350 - nse: 0.9173 - val_loss: 0.0930 - val_mean_absolute_error: 0.1969 - val_nse: 0.7261
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.0348 - mean_absolute_error: 0.1337 - nse: 0.9226
Epoch 00047: val_nse did not improve from 0.86233
10/10 [==============================] - 8s 789ms/step - loss: 0.0348 - mean_absolute_error: 0.1337 - nse: 0.9226 - val_loss: 0.1034

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
